# Pokemon Image Classification

### Preprocessing Image Data

In [8]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import os

# Directory paths
pdf_dir = 'img'
output_dir = 'output_images'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to dynamically trim the left side
def trim_left_side(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection
    edges = cv2.Canny(gray, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the leftmost contour point
    min_x = image.shape[1]  # Initialize with the max possible width
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x < min_x:
            min_x = x
    
    # Add a small margin to min_x
    margin = 10
    trim_x = min_x + margin

    # Trim the image
    trimmed_image = image[:, trim_x:]
    return trimmed_image

# Iterate through each PDF file in the directory
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, pdf_file)

        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Iterate through each page in the PDF
        for page_num in range(len(pdf_document)):
            # Get the page
            page = pdf_document.load_page(page_num)

            # Convert the page to an image using PyMuPDF
            pix = page.get_pixmap()
            img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # Trim the left side dynamically
            img_trimmed = trim_left_side(img)

            # Get trimmed image dimensions
            img_height, img_width, _ = img_trimmed.shape

            # Define the dimensions of each smaller image
            card_width = img_width // 3
            card_height = img_height // 3

            # Split the image into 9 individual images
            cards = []
            for row in range(3):
                for col in range(3):
                    left = col * card_width
                    upper = row * card_height
                    right = left + card_width
                    lower = upper + card_height
                    card = img_trimmed[upper:lower, left:right]
                    cards.append(card)

            # Save each individual image
            base_filename = os.path.splitext(pdf_file)[0]
            for idx, card in enumerate(cards):
                output_filename = f'{output_dir}/{base_filename}_page_{page_num + 1}_card_{idx + 1}.png'
                cv2.imwrite(output_filename, card)

print("Images have been successfully split and saved from all PDFs.")


Images have been successfully split and saved from all PDFs.


### Pokémon TCG API

In [6]:
# Load .env file for Pokémon TCG API
from dotenv import load_dotenv
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity
import json

load_dotenv()

cards = Card.all()

# Convert the list of card objects to a list of dictionaries
cards_dict = [card.__dict__ for card in cards]

# Custom serialization function
def custom_serializer(obj):
    if hasattr(obj, '__dict__'):
        return obj.__dict__
    else:
        return str(obj)

# Open the file in write mode and dump the JSON data
with open("data/all_cards.json", "w") as file:
    json.dump(cards_dict, file, default=custom_serializer)

In [2]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import os

# File paths
pdf_file = 'img/two.pdf'
output_dir = 'output_images'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Open the PDF file
pdf_document = fitz.open(pdf_file)

# Iterate through each page in the PDF
for page_num in range(len(pdf_document)):
    # Get the page
    page = pdf_document.load_page(page_num)

    # Convert the page to an image using PyMuPDF
    pix = page.get_pixmap()
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Save the image
    output_filename = f'{output_dir}/one_page_{page_num + 1}.png'
    cv2.imwrite(output_filename, img)

print("Images have been successfully converted and saved from one.pdf.")

Images have been successfully converted and saved from one.pdf.
